### 数据滤波与提取
> 这一步首先对原始数据进行滤波，然后通过步长为128的窗口以50%的重叠率把每次采集的数据整理成若干条数据，再对每一条数据进行特征提取转化成可以作为模型输入的features。

tips: 微信小程序数据库中直接导出的是若干500\*6的.csv表格，每一行代表记录的时刻（以20ms为间隔，从1～500编号），每一列代表某传感器某一轴的测量值（ax,ay,az,gx,gy,gz），每一张表格以“ai-j.csv”命名，表示动作i的第j次数据采集。

In [1]:
# 首先导入必要的包
import numpy as np
from scipy import signal as signal

# 声明待处理的文件路径，批量化处理时更改此处即可，提高模块化程度
data_No = './data/a1-5'

接下来定义三个后面可能用到的函数

In [2]:
# 把数据以50%的overlap进行拆分，每个窗口长度为128个（2.56秒），同时分出六个轴
# 输出：6个6*128的表格
def spl(data):
    ax = np.zeros((6,128))
    for i in range(6):
        ax[i,:] = data[0,i*64:i*64+128]
    ay = np.zeros((6,128))
    for i in range(6):
        ay[i,:] = data[1,i*64:i*64+128]
    az = np.zeros((6,128))
    for i in range(6):
        az[i,:] = data[2,i*64:i*64+128]
    gx = np.zeros((6,128))
    for i in range(6):
        gx[i,:] = data[3,i*64:i*64+128]
    gy = np.zeros((6,128))
    for i in range(6):
        gy[i,:] = data[4,i*64:i*64+128]
    gz = np.zeros((6,128))
    for i in range(6):
        gz[i,:] = data[5,i*64:i*64+128]
    return ax,ay,az,gx,gy,gz

In [3]:
# 滤波
def filtering(data):
    rows = data.shape[0]
# 中值滤波
    for i in range(rows):
        data[i,:] = signal.medfilt(data[i,:],3)
# 巴特沃斯低通滤波
    b, a = signal.butter(3, 0.8, 'lowpass')
    for i in range(rows):
        data[i,:] = signal.filtfilt(b,a,data[i,:])
    return data

In [4]:
# 生成特征值
# 这一步需要根据试验情况调整生成特征值的种类
# 实践中发现，过多的特征值会导致模型计算中有一步出现overflow的情况，同时不利于小程序的部署
# 因此特征选择尽量少且具有代表性
def generate_feature(data):
    f = np.zeros((6,1))
    f[:,0] = np.mean(data,axis=1)
    return f

In [5]:
# 对具体数据进行操作
df = np.loadtxt(data_No+'.csv',skiprows=1,usecols = (1,2,3,4,5,6),delimiter = ',')  # 首先把数据读进来（跳过第一列的编号）
filtered = filtering(df.T)  # 把表格转置一下然后滤波
ax,ay,az,gx,gy,gz = spl(filtered)  # 按照步长128重叠率50%进行拆分

In [6]:
# 将上面得到的处理完的数据进行操作得到features
fax = generate_feature(ax)
fay = generate_feature(ay)
faz = generate_feature(az)
fgx = generate_feature(gx)
fgy = generate_feature(gy)
fgz = generate_feature(gz)

# 把六个轴得到的features整合到一张表中
F = np.hstack((fax,fay,faz,fgx,fgy,fgz))

In [7]:
# 导出最后生成的features表格
np.savetxt(data_No+'features.csv',F,delimiter = ',')

### 模型构建与训练
> 模型部分选用的是朴素贝叶斯分类器(Naive Bayes Classifier)，选用该算法是基于一个假设，即不同人做同一动作的体态应该服从高斯分布。

In [8]:
# 首先导入必要的包
import numpy as np

# 加载features和labels表格
X_train = np.loadtxt('features.csv',usecols = (0,1,2,3,4,5),delimiter = ',')
# X_train = np.loadtxt('features.txt')
Y_train = np.loadtxt('targets.txt',dtype=int)
data_dim = X_train.shape[1]

In [9]:
# 按照所属类别对训练数据进行拆分
X_train_0 = np.array([x for x, y in zip(X_train, Y_train) if y == 0])
X_train_1 = np.array([x for x, y in zip(X_train, Y_train) if y == 1])
X_train_2 = np.array([x for x, y in zip(X_train, Y_train) if y == 2])
X_train_3 = np.array([x for x, y in zip(X_train, Y_train) if y == 3])

In [10]:
# 计算每一个类别的类内均值
mean_0 = np.mean(X_train_0, axis = 0)
mean_1 = np.mean(X_train_1, axis = 0)
mean_2 = np.mean(X_train_2, axis = 0)
mean_3 = np.mean(X_train_3, axis = 0)

In [11]:
# 计算每一个类别的类内方差
cov_0 = np.zeros((data_dim, data_dim))
cov_1 = np.zeros((data_dim, data_dim))
cov_2 = np.zeros((data_dim, data_dim))
cov_3 = np.zeros((data_dim, data_dim))

for x in X_train_0:
    cov_0 += np.dot(np.transpose([x - mean_0]), [x - mean_0]) / X_train_0.shape[0]
for x in X_train_1:
    cov_1 += np.dot(np.transpose([x - mean_1]), [x - mean_1]) / X_train_1.shape[0]
for x in X_train_2:
    cov_2 += np.dot(np.transpose([x - mean_2]), [x - mean_2]) / X_train_2.shape[0]
for x in X_train_3:
    cov_3 += np.dot(np.transpose([x - mean_3]), [x - mean_3]) / X_train_3.shape[0]

#### ！！！高能预警！！！

以下部分可以说是模型最关键的地方，计算具体的权重w和偏差b。

在对高斯分布的公式进行化简的时候，假设当前计算的两类具有相同类内方差，该方差为两类各自方差的加权平均，这个假设是为了减少模型中的参数，一方面增加模型的范化能力，另一方面方便模型在小程序上的部署。具体化简过程需要一定数学基础，在此直接呈现结果。

下面的步骤一共需要重复六次，因为一共有四类，两两之间都要计算一次

In [12]:
# 每两类共用一个方差，该方差为这两类方差的加权平均，下同
cov_01 = (cov_0 * X_train_0.shape[0] + cov_1 * X_train_1.shape[0]) / (X_train_0.shape[0] + X_train_1.shape[0])
u, s, v = np.linalg.svd(cov_01, full_matrices=False)
inv = np.matmul(v.T * 1 / s, u.T)
# 由此可以直接计算出权重和偏差，下同
w01 = np.dot(inv, mean_0 - mean_1)
b01 =  (-0.5) * np.dot(mean_0, np.dot(inv, mean_0)) + 0.5 * np.dot(mean_1, np.dot(inv, mean_1))\
    + np.log(float(X_train_0.shape[0]) / X_train_1.shape[0])

cov_02 = (cov_0 * X_train_0.shape[0] + cov_2 * X_train_2.shape[0]) / (X_train_0.shape[0] + X_train_2.shape[0])
u, s, v = np.linalg.svd(cov_02, full_matrices=False)
inv = np.matmul(v.T * 1 / s, u.T)
w02 = np.dot(inv, mean_0 - mean_2)
b02 =  (-0.5) * np.dot(mean_0, np.dot(inv, mean_0)) + 0.5 * np.dot(mean_2, np.dot(inv, mean_2))\
    + np.log(float(X_train_0.shape[0]) / X_train_2.shape[0]) 

cov_03 = (cov_0 * X_train_0.shape[0] + cov_3 * X_train_3.shape[0]) / (X_train_0.shape[0] + X_train_3.shape[0])
u, s, v = np.linalg.svd(cov_03, full_matrices=False)
inv = np.matmul(v.T * 1 / s, u.T)
w03 = np.dot(inv, mean_0 - mean_3)
b03 =  (-0.5) * np.dot(mean_0, np.dot(inv, mean_0)) + 0.5 * np.dot(mean_3, np.dot(inv, mean_3))\
    + np.log(float(X_train_0.shape[0]) / X_train_3.shape[0])

cov_12 = (cov_1 * X_train_1.shape[0] + cov_2 * X_train_2.shape[0]) / (X_train_1.shape[0] + X_train_2.shape[0])
u, s, v = np.linalg.svd(cov_12, full_matrices=False)
inv = np.matmul(v.T * 1 / s, u.T)
w12 = np.dot(inv, mean_1 - mean_2)
b12 =  (-0.5) * np.dot(mean_1, np.dot(inv, mean_1)) + 0.5 * np.dot(mean_2, np.dot(inv, mean_2))\
    + np.log(float(X_train_1.shape[0]) / X_train_2.shape[0]) 

cov_13 = (cov_1 * X_train_1.shape[0] + cov_3 * X_train_3.shape[0]) / (X_train_1.shape[0] + X_train_3.shape[0])
u, s, v = np.linalg.svd(cov_13, full_matrices=False)
inv = np.matmul(v.T * 1 / s, u.T)
w13 = np.dot(inv, mean_1 - mean_3)
b13 =  (-0.5) * np.dot(mean_1, np.dot(inv, mean_1)) + 0.5 * np.dot(mean_3, np.dot(inv, mean_3))\
    + np.log(float(X_train_1.shape[0]) / X_train_3.shape[0])

cov_23 = (cov_2 * X_train_2.shape[0] + cov_3 * X_train_3.shape[0]) / (X_train_2.shape[0] + X_train_3.shape[0])
u, s, v = np.linalg.svd(cov_23, full_matrices=False)
inv = np.matmul(v.T * 1 / s, u.T)
w23 = np.dot(inv, mean_2 - mean_3)
b23 =  (-0.5) * np.dot(mean_2, np.dot(inv, mean_2)) + 0.5 * np.dot(mean_3, np.dot(inv, mean_3))\
    + np.log(float(X_train_2.shape[0]) / X_train_3.shape[0]) 

In [13]:
# 输出模型中的参数，便于移植到小程序上
print("w01:",w01,"\nb01:",b01,"\n\nw02:",w02,"\nb02:",b02,"\n\nw03:",w03,"\nb03:",b03)
print("\n\nw12:",w12,"\nb12:",b12,"\n\nw13:",w13,"\nb13:",b13,"\n\nw23:",w23,"\nb23:",b23)

w01: [-2.66101376 41.79562253  7.66104293  2.64022394  0.33635261 -9.00248786] 
b01: -8.295051012701379 

w02: [-81.4939717   34.61763437 263.18878502 -27.20539051  61.15050547
  16.38442745] 
b02: 190.80706834346483 

w03: [ 62.97536227  40.78850165  -5.6585328   -4.64453211   7.78903076
 -21.49121195] 
b03: -70.49755657118743


w12: [ 2.39555501 -7.88607327 12.58942167 14.10338539 10.07135474 10.2806766 ] 
b12: 8.545704376265395 

w13: [ 21.8592652    4.83530056  -7.05969533  -4.35725167  -3.82293505
 -10.29726871] 
b13: -21.240499109311852 

w23: [ 11.99353123  22.69249114 -50.7814112  -10.79888599 -26.81164328
 -14.07464918] 
b23: -45.22007401575592


In [14]:
# 定义预测函数，得到样本属于每一类的概率值，并输出它最可能属于的类别
# 在小程序中概率的具体计算同下面公式
def predict(x):
    z01 = np.dot(x,w01) + b01
    z02 = np.dot(x,w02) + b02
    z03 = np.dot(x,w03) + b03
    z12 = np.dot(x,w12) + b12
    z13 = np.dot(x,w13) + b13
    z23 = np.dot(x,w23) + b23
    
    p = np.zeros((4))
    p[0] = 1 / (1 + np.exp(-z01)+ np.exp(-z02)+ np.exp(-z03))
    p[1] = 1 / (1 + np.exp(z01) + np.exp(-z12)+ np.exp(-z13))
    p[2] = 1 / (1 + np.exp(z02) + np.exp(z12) + np.exp(-z23))
    p[3] = 1 / (1 + np.exp(z03) + np.exp(z13) + np.exp(z23))
    
    #print(p)
    print(np.argmax(p))